In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from torchvision import transforms, datasets

from utils_plotting import *

## Dataset

In [ ]:
# Regular NN

batch_size = 128
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
valset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,num_workers=3)

x_dev = []
y_dev = []
for x, y in valloader:
    x_dev.append(x.cpu().numpy())
    y_dev.append(y.cpu().numpy())

x_dev = np.concatenate(x_dev)
y_dev = np.concatenate(y_dev)
print(x_dev.shape)
print(y_dev.shape)

## Predicitve accuracy VS Rotation angle

In [ ]:
result_name_list = ["Results/Regular_results"]

correct_preds = np.load(result_name_list[0] + '/correct_preds.npy')
all_preds = np.load(result_name_list[0] + '/all_preds.npy')

pred_entropy = -(all_preds * np.log(all_preds)).sum(axis=2)
pred_entropy_mean = pred_entropy.mean(axis=0)
pred_entropy_std = pred_entropy.std(axis=0)

# Get correct prediction probabilities
correct_mean = correct_preds.mean(axis=0)
correct_std = correct_preds.std(axis=0)

In [ ]:
s_rot = 0
end_rot = 179
steps = 16
rotations = (np.linspace(s_rot, end_rot, steps)).astype(int)

fs = 25

# plt.figure(dpi=100)
# ax = plt.gca()
fig = plt.figure(figsize=(steps, 12), dpi=80)

ax = plt.subplot2grid((3, steps-1), (0, 0), rowspan=2, colspan=steps-1)


ax2 = ax.twinx()

aa, = errorfill(rotations, correct_mean, yerr=correct_std, color=c[2], alpha_fill=0.25, ax=ax)

#     ax = plt.gca()
ax.set_xlabel('rotation angle')
ax.set_ylabel('predictive accuracy')
ax.set_title('MAP')
ax2.yaxis.grid()
ax.xaxis.grid() 


ax2.set_ylabel('nats')
# bb, = errorfill(rotations, total_entropy_mean, yerr=total_entropy_std, color=c[0], ax=ax2)
cc, = errorfill(rotations, pred_entropy_mean, yerr=pred_entropy_std, color=c[1], ax=ax2)
#     dd, = errorfill(rotations, epistemic_entropy_mean, yerr=epistemic_entropy_std, color=c[3], ax=ax2)


# lgd = plt.legend(['correct class', 'posterior predictive entropy'], loc='upper right',
#                  prop={'size': 15, 'weight': 'normal'}, bbox_to_anchor=(1.4,1))

p_lgd = [aa, cc]
lgd = plt.legend(p_lgd,
['$p(y = t\,|\, x, \ w)$', '$\mathcal{H}(y\,|\, x, w)$'],
                prop={'size': 25, 'weight': 'normal'})


plt.autoscale(enable=True, axis='x', tight=True)

Nim = 82
s_rot = 0
end_rot = 179
steps = 10
rotations = (np.linspace(s_rot, end_rot, steps)).astype(int)

for i in range(1,len(rotations)):
    angle = rotations[i]
    # x_rot = ndim.interpolation.rotate(x_dev[Nim, :, :, :], angle, axes = (1,2), reshape=False)
    x_rot = ndim.interpolation.rotate(valset.data[Nim], angle, axes = (0,1), reshape=False, mode='nearest')
    ax3 = fig.add_subplot(3, (steps-1), 2*(steps-1)+i)
    # ax3.imshow(np.transpose(x_rot,(1,2,0)))
    ax3.imshow(x_rot)
    ax3.axis('off')
    ax3.set_xticklabels([])
    ax3.set_yticklabels([])

    
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + [ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
            ax.get_xticklabels() + ax.get_yticklabels() + ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(fs)
    item.set_weight('normal')

plt.autoscale(enable=True, axis='x', tight=True)
ax.set_ylim(bottom=0, top=1)
ax2.set_ylim(bottom=0, top=2)


plt.show()


## Entropy curves

In [ ]:
# Check shapes of files

dirname = 'Results/BBP_results/bbb'
correct_preds = np.load(dirname + '/correct_preds.npy')
all_preds = np.load(dirname + '/all_preds.npy')
all_sample_preds = np.load(dirname + '/all_sample_preds.npy')

print(correct_preds.shape, all_preds.shape, all_sample_preds.shape)

In [ ]:
result_name_list = ['Results/Regular_results', 'Results/MCdrop_results', 'Results/Ensemble_results']
names = ['Regular NN', 'MC Dropout', 'Boostrap Ensemble']

s_rot = 0
end_rot = 179
steps = 16
rotations = (np.linspace(s_rot, end_rot, steps)).astype(int)

# plt.figure(dpi=120)

fs = 15
Ncols = len(result_name_list)

f, ax_vec = plt.subplots(1, Ncols, figsize=(6.5*Ncols, 5)) # , sharey=True


for idx, dirname in enumerate(result_name_list):
    
    ax = ax_vec[idx]
    ax2 = ax.twinx()

    correct_preds = np.load(dirname + '/correct_preds.npy')
    all_preds = np.load(dirname + '/all_preds.npy')
    all_sample_preds = np.load(dirname + '/all_sample_preds.npy')
    all_sample_preds = all_sample_preds + 1e-5

    # print(correct_preds.shape)
    # print(all_sample_preds.shape)

    # Get correct prediction probabilities
    correct_mean = correct_preds.mean(axis=0)
    correct_std = correct_preds.std(axis=0)

    # Compute the approx posterior predictive
    posterior_preds = all_sample_preds.mean(axis=1)
    total_entropy = -(posterior_preds * np.log(posterior_preds)).sum(axis=2)
    total_entropy_mean = total_entropy.mean(axis=0)
    total_entropy_std = total_entropy.std(axis=0)

    # Get sample wise metrics (entropy) -> aleatoric entropy
    sample_preds_entropy = -( all_sample_preds * np.log(all_sample_preds) ).sum(axis=3)
    aleatoric_entropy = sample_preds_entropy.mean(axis=1)
    aleatoric_entropy_mean = aleatoric_entropy.mean(axis=0)
    aleatoric_entropy_std = aleatoric_entropy.std(axis=0)

    # Get epistemic entropy 
    epistemic_entropy = total_entropy - aleatoric_entropy
    epistemic_entropy_mean = epistemic_entropy.mean(axis=0)
    epistemic_entropy_std = epistemic_entropy.std(axis=0)

    # print('expected entropy mean', aleatoric_entropy_mean.shape)
    # print('posterior entropy mean', posterior_mean_angle_entropy.shape)


    aa, = errorfill(rotations, correct_mean, yerr=correct_std, color=c[2], alpha_fill=0.25, ax=ax)

#     ax = plt.gca()
    ax.set_xlabel('rotation angle')
    ax.set_ylabel('predictive accuracy')
    ax.set_title('%s' % names[idx])
    ax.yaxis.grid() 
    ax.xaxis.grid()

    
    ax2.set_ylabel('nats')
    bb, = errorfill(rotations, total_entropy_mean, yerr=total_entropy_std, color=c[0], ax=ax2)
    cc, = errorfill(rotations, aleatoric_entropy_mean, yerr=aleatoric_entropy_std, color=c[1], ax=ax2)
    dd, = errorfill(rotations, epistemic_entropy_mean, yerr=epistemic_entropy_std, color=c[3], ax=ax2)


    # lgd = plt.legend(['correct class', 'posterior predictive entropy'], loc='upper right',
    #                  prop={'size': 15, 'weight': 'normal'}, bbox_to_anchor=(1.4,1))


    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + [ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels() + ax2.get_xticklabels() + ax2.get_yticklabels()):
        item.set_fontsize(fs)
        item.set_weight('normal')

    plt.autoscale(enable=True, axis='x', tight=True)
    ax.set_ylim(bottom=0, top=1)
    ax2.set_ylim(bottom=0, top=2)

p_lgd = [aa, bb, cc, dd]
lgd = plt.legend(p_lgd, ['$p(Y = t\,|\,x, w)$', '$\mathcal{H}(y\,|\,x)$', '$E_{q(w)}[\mathcal{H}(y\,|\,x, w)]$', '$\mathcal{H}(\mathbb{E}) - \mathbb{E}[\mathcal{H}]$'], bbox_to_anchor=(1.7,1))

plt.tight_layout()

In [ ]:
result_name_list = ['Results/BBP_results/bbb', 'Results/BBP_results/lrt', 'Results/Contrastive_Reasoning_results', 'Results/Tent_results']
names = ['BBP:Without lrt' , 'BBP: With lrt', 'Contrastive Reasoning', 'TENT']

s_rot = 0
end_rot = 179
steps = 16
rotations = (np.linspace(s_rot, end_rot, steps)).astype(int)

# plt.figure(dpi=120)

fs = 15
Ncols = len(result_name_list)

f, ax_vec = plt.subplots(1, Ncols, figsize=(6.5*Ncols, 5)) # , sharey=True


for idx, dirname in enumerate(result_name_list):
    
    ax = ax_vec[idx]
    ax2 = ax.twinx()

    correct_preds = np.load(dirname + '/correct_preds.npy')
    all_preds = np.load(dirname + '/all_preds.npy')
    all_sample_preds = np.load(dirname + '/all_sample_preds.npy')
    all_sample_preds = all_sample_preds + 1e-5

    # print(correct_preds.shape)
    # print(all_sample_preds.shape)

    # Get correct prediction probabilities
    correct_mean = correct_preds.mean(axis=0)
    correct_std = correct_preds.std(axis=0)

    # Compute the approx posterior predictive
    posterior_preds = all_sample_preds.mean(axis=1)
    total_entropy = -(posterior_preds * np.log(posterior_preds)).sum(axis=2)
    total_entropy_mean = total_entropy.mean(axis=0)
    total_entropy_std = total_entropy.std(axis=0)

    # Get sample wise metrics (entropy) -> aleatoric entropy
    sample_preds_entropy = -( all_sample_preds * np.log(all_sample_preds) ).sum(axis=3)
    aleatoric_entropy = sample_preds_entropy.mean(axis=1)
    aleatoric_entropy_mean = aleatoric_entropy.mean(axis=0)
    aleatoric_entropy_std = aleatoric_entropy.std(axis=0)

    # Get epistemic entropy 
    epistemic_entropy = total_entropy - aleatoric_entropy
    epistemic_entropy_mean = epistemic_entropy.mean(axis=0)
    epistemic_entropy_std = epistemic_entropy.std(axis=0)

    # print('expected entropy mean', aleatoric_entropy_mean.shape)
    # print('posterior entropy mean', posterior_mean_angle_entropy.shape)


    aa, = errorfill(rotations, correct_mean, yerr=correct_std, color=c[2], alpha_fill=0.25, ax=ax)

#     ax = plt.gca()
    ax.set_xlabel('rotation angle')
    ax.set_ylabel('predictive accuracy')
    ax.set_title('%s' % names[idx])
    ax.yaxis.grid() 
    ax.xaxis.grid()

    
    ax2.set_ylabel('nats')
    bb, = errorfill(rotations, total_entropy_mean, yerr=total_entropy_std, color=c[0], ax=ax2)
    cc, = errorfill(rotations, aleatoric_entropy_mean, yerr=aleatoric_entropy_std, color=c[1], ax=ax2)
    dd, = errorfill(rotations, epistemic_entropy_mean, yerr=epistemic_entropy_std, color=c[3], ax=ax2)


    # lgd = plt.legend(['correct class', 'posterior predictive entropy'], loc='upper right',
    #                  prop={'size': 15, 'weight': 'normal'}, bbox_to_anchor=(1.4,1))


    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + [ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels() + ax2.get_xticklabels() + ax2.get_yticklabels()):
        item.set_fontsize(fs)
        item.set_weight('normal')

    plt.autoscale(enable=True, axis='x', tight=True)
    ax.set_ylim(bottom=0, top=1)
    ax2.set_ylim(bottom=0, top=2)

    
p_lgd = [aa, bb, cc, dd]
lgd = plt.legend(p_lgd, ['$p(Y = t\,|\,x, w)$', '$\mathcal{H}(y\,|\,x)$', '$E_{q(w)}[\mathcal{H}(y\,|\,x, w)]$', '$\mathcal{H}(\mathbb{E}) - \mathbb{E}[\mathcal{H}]$'], bbox_to_anchor=(1.7,1))

plt.tight_layout()

In [ ]:
def plot_predictive_entropy(correct_preds, all_preds, rotations):
    all_preds_entropy = -(all_preds * np.log(all_preds)).sum(axis=2)
    mean_angle_entropy = all_preds_entropy.mean(axis=0)
    std_angle_entropy = all_preds_entropy.std(axis=0)

    correct_mean = correct_preds.mean(axis=0)
    correct_std = correct_preds.std(axis=0)

    # plt.figure(dpi=100)
    line_ax0 = errorfill(rotations, correct_mean, yerr=correct_std, color=c[2])
    ax = plt.gca()
    ax2 = ax.twinx()
    line_ax1 = errorfill(rotations, mean_angle_entropy, yerr=std_angle_entropy, color=c[3], ax=ax2)
    plt.xlabel('rotation angle')
    lns = line_ax0+line_ax1

    # lgd = plt.legend(lns, ['correct class', 'predictive entropy'], loc='upper right',
    #                 prop={'size': 15, 'weight': 'normal'}, bbox_to_anchor=(1.75,1))


    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + [ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels() + ax2.get_xticklabels() + ax2.get_yticklabels()):
        item.set_fontsize(12)
        item.set_weight('normal')
    plt.autoscale(enable=True, axis='x', tight=True)

result_name_list = ['Results/Regular_results', 'Results/MCdrop_results', 'Results/Ensemble_results',
                     'Results/BBP_results/bbb', 'Results/BBP_results/lrt', 'Results/GradCon_results']
names = ["Regular", 'MC Dropout', 'Bootstrap Ensemble', 'BBP: without lrt' , 'BBP: with lrt', 'GradCon']

steps = 16
rotations = (np.linspace(0, 179, steps)).astype(int)

plt.figure(figsize=(20,10))
for n,dir in enumerate(result_name_list):
    correct_preds = np.load(dir + '/correct_preds.npy')
    all_preds = np.load(dir + '/all_preds.npy')
    
    plt.subplot(2,3,n+1)
    plt.title(names[n])
    plot_predictive_entropy(correct_preds, all_preds, rotations)

# lgd = plt.legend(['correct class', 'predictive entropy'], loc='upper right',
#                     prop={'size': 15, 'weight': 'normal'}, bbox_to_anchor=(1.75,1))

In [ ]:
dirname = 'Results/MCdrop_results'

correct_preds = np.load(dirname + '/correct_preds.npy')
# all_preds = np.load(dirname + '/all_preds.npy')
all_sample_preds = np.load(dirname + '/all_sample_preds.npy')
all_sample_preds = all_sample_preds + 1e-5

# print(correct_preds.shape)
# print(all_sample_preds.shape)

# Get correct prediction probabilities
correct_mean = correct_preds.mean(axis=0)
correct_std = correct_preds.std(axis=0)
# print(correct_preds.shape)

print(all_sample_preds.shape)
# Total entropy
posterior_preds = all_sample_preds.mean(axis=1) # Mean across 100 samples
total_entropy = -(posterior_preds * np.log(posterior_preds)).sum(axis=2) # Entropy
total_entropy_mean = total_entropy.mean(axis=0) 
total_entropy_std = total_entropy.std(axis=0)
print(total_entropy.shape)

# Aleatoric entropy
sample_preds_entropy = -( all_sample_preds * np.log(all_sample_preds) ).sum(axis=3) # Entropy
aleatoric_entropy = sample_preds_entropy.mean(axis=1) # Mean across 100 samples
aleatoric_entropy_mean = aleatoric_entropy.mean(axis=0)
aleatoric_entropy_std = aleatoric_entropy.std(axis=0)
print(aleatoric_entropy.shape)

# Epistemic entropy 
epistemic_entropy = total_entropy - aleatoric_entropy # Difference
epistemic_entropy_mean = epistemic_entropy.mean(axis=0)
epistemic_entropy_std = epistemic_entropy.std(axis=0)
print(epistemic_entropy.shape)

## Calibration curves

### CIFAR-10

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

result_name_list = ['Results/Regular_results', 'Results/MCdrop_results', 'Results/Ensemble_results',
                     'Results/BBP_results/bbb', 'Results/BBP_results/lrt', 'Results/Contrastive_Reasoning_results', 'Results/Tent_results']
names = ["Regular", 'MC Dropout', 'Bootstrap Ensemble', 'BBP: without lrt' , 'BBP: with lrt', 'Contrastive Reasoning', 'TENT']

bin_step = 0.1
bins = np.arange(0, 1.0001, bin_step)

fig = plt.figure(dpi=120)
ax = fig.add_subplot(111)

min_idx = 10

for dir_idx, dirname in enumerate(result_name_list):
    

    targets = y_dev
    all_preds = np.load(dirname + '/all_preds.npy')


    all_preds = all_preds[:, 0, :]
    pred_class = np.argmax(all_preds, axis=1)
    # targets = targets[:,0]
    # targets = np.around(targets).astype(int)

    expanded_preds = np.reshape(all_preds, -1)
    pred_class_OH_expand = np.reshape(get_one_hot(pred_class, 10), -1)
    targets_class_OH_expand = np.reshape(get_one_hot(targets, 10), -1)
    
    correct_vec = (targets_class_OH_expand*(pred_class_OH_expand == targets_class_OH_expand)).astype(int)
#     correct_vec = (pred_class == targets).astype(int)

    bin_idxs = np.digitize(expanded_preds, bins, right=True)
    bin_idxs = bin_idxs - 1

    bin_centers = bins[1:] - bin_step/2

    bin_counts = np.ones(len(bin_centers))
    bin_corrects = np.zeros(len(bin_centers))
    
#     print(min(bin_idxs))
    if min(bin_idxs) < min_idx:
        min_idx = min(bin_idxs)

    for nbin in range(len(bin_centers)):

        bin_counts[nbin] = np.sum((bin_idxs==nbin).astype(int))
        bin_corrects[nbin] = np.sum(correct_vec[bin_idxs==nbin])

    have_data = bin_counts > 0  
    bin_pcorr = bin_corrects[have_data] / bin_counts[have_data]

    ax.plot(bin_centers[have_data], bin_pcorr, '-o', label=names[dir_idx])

# print(min_idx)

ax.legend()
plt.tight_layout()

ax.plot(bin_centers[min_idx:], bin_centers[min_idx:], '--', c='k')
ax.set_xticks(bin_centers[min_idx:]) # [labels]

ax.set_xlabel('Predicted probability')
ax.set_ylabel('Correct proportion')
ax.set_title('Calibration Curve')

### CIFAR-10 Rotations

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

result_name_list = ['Results/Regular_results', 'Results/MCdrop_results', 'Results/Ensemble_results',
                     'Results/BBP_results/bbb', 'Results/BBP_results/lrt', 'Results/Contrastive_Reasoning_results', 'Results/Tent_results']
names = ["Regular", 'MC Dropout', 'Bootstrap Ensemble', 'BBP: without lrt' , 'BBP: with lrt', 'Contrastive Reasoning', 'TENT']
bin_step = 0.1
bins = np.arange(0, 1.0001, bin_step)

fig = plt.figure(dpi=120)
ax = fig.add_subplot(111)

min_idx = 10
targets = np.repeat(y_dev,16)

for dir_idx, dirname in enumerate(result_name_list):
    
    all_preds = np.load(dirname + '/all_preds.npy')
    N, R, C = all_preds.shape
    all_preds = all_preds.reshape(-1, C) # Over all rotations
    pred_class = np.argmax(all_preds, axis=1)
    # targets = targets[:,0]
    # targets = np.around(targets).astype(int)

    expanded_preds = np.reshape(all_preds, -1)
    pred_class_OH_expand = np.reshape(get_one_hot(pred_class, 10), -1)
    targets_class_OH_expand = np.reshape(get_one_hot(targets, 10), -1)
    
    correct_vec = (targets_class_OH_expand*(pred_class_OH_expand == targets_class_OH_expand)).astype(int)
#     correct_vec = (pred_class == targets).astype(int)

    bin_idxs = np.digitize(expanded_preds, bins, right=True)
    bin_idxs = bin_idxs - 1

    bin_centers = bins[1:] - bin_step/2

    bin_counts = np.ones(len(bin_centers))
    bin_corrects = np.zeros(len(bin_centers))
    
#     print(min(bin_idxs))
    if min(bin_idxs) < min_idx:
        min_idx = min(bin_idxs)

    for nbin in range(len(bin_centers)):

        bin_counts[nbin] = np.sum((bin_idxs==nbin).astype(int))
        bin_corrects[nbin] = np.sum(correct_vec[bin_idxs==nbin])

    have_data = bin_counts > 0  
    bin_pcorr = bin_corrects[have_data] / bin_counts[have_data]

    ax.plot(bin_centers[have_data], bin_pcorr, '-o', label=names[dir_idx])

# print(min_idx)

ax.legend()
plt.tight_layout()

ax.plot(bin_centers[min_idx:], bin_centers[min_idx:], '--', c='k')
ax.set_xticks(bin_centers[min_idx:]) # [labels]

ax.set_xlabel('Predicted probability')
ax.set_ylabel('Correct proportion')
ax.set_title('Calibration Curve')
plt.savefig('cc_cifar10rot.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')

### CIFAR-10-C

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

result_name_list = ['Results/Regular_results', 'Results/MCdrop_results', 'Results/Ensemble_results',
                     'Results/BBP_results/bbb', 'Results/BBP_results/lrt', 'Results/Contrastive_Reasoning_results', 'Results/Tent_results']
names = ["Regular", 'MC Dropout', 'Bootstrap Ensemble', 'BBP: without lrt' , 'BBP: with lrt', 'Contrastive Reasoning', 'TENT']

bin_step = 0.1
bins = np.arange(0, 1.0001, bin_step)

fig = plt.figure(dpi=120)
ax = fig.add_subplot(111)

min_idx = 10
targets = np.tile(y_dev,5*19)

for dir_idx, dirname in enumerate(result_name_list):
    

    all_preds = np.load(dirname + '/preds_CIFAR-10-C.npy')


    # all_preds = all_preds[:, 0, :]
    pred_class = np.argmax(all_preds, axis=1)
    # targets = targets[:,0]
    # targets = np.around(targets).astype(int)

    expanded_preds = np.reshape(all_preds, -1)
    pred_class_OH_expand = np.reshape(get_one_hot(pred_class, 10), -1)
    targets_class_OH_expand = np.reshape(get_one_hot(targets, 10), -1)
    
    correct_vec = (targets_class_OH_expand*(pred_class_OH_expand == targets_class_OH_expand)).astype(int)
#     correct_vec = (pred_class == targets).astype(int)

    bin_idxs = np.digitize(expanded_preds, bins, right=True)
    bin_idxs = bin_idxs - 1

    bin_centers = bins[1:] - bin_step/2

    bin_counts = np.ones(len(bin_centers))
    bin_corrects = np.zeros(len(bin_centers))
    
#     print(min(bin_idxs))
    if min(bin_idxs) < min_idx:
        min_idx = min(bin_idxs)

    for nbin in range(len(bin_centers)):

        bin_counts[nbin] = np.sum((bin_idxs==nbin).astype(int))
        bin_corrects[nbin] = np.sum(correct_vec[bin_idxs==nbin])

    have_data = bin_counts > 0  
    bin_pcorr = bin_corrects[have_data] / bin_counts[have_data]

    ax.plot(bin_centers[have_data], bin_pcorr, '-o', label=names[dir_idx])

# print(min_idx)

ax.legend()
plt.tight_layout()

ax.plot(bin_centers[min_idx:], bin_centers[min_idx:], '--', c='k')
ax.set_xticks(bin_centers[min_idx:]) # [labels]

ax.set_xlabel('Predicted probability')
ax.set_ylabel('Correct proportion')
ax.set_title('Calibration Curve')
plt.savefig('cc_cifar10c.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')